## Homework 4 ##

### Question 2: Max-flow problem formulation as an LP problem

Consider a directed graph $G = (V,E)$, where $V$ denotes a set of vertices and $E \subseteq V \times V$ denotes a set of edges. Let $e = (u, v) \in E$ be an edge for vertex $u$ to vertex $v$, we define $e \in \delta^+ (v)$ to be the set of all outgoing edges to vertex $v$ and $e \in \delta^- (v)$ to be the set of all incoming nodes from vertex $v$. Let $c(e), \ e \in E$ be the capacity of the edge, i.e., the maximum amount of commodity that one can push through the edge. Let $f(e)$ be the flow across an edge $e$. $f(e)$ is non-negative as the flow in the edge cannot flow in the reverse direction i.e., towards the source. Also, the flow across each node should be conserved. In other words, the flow coming into a node has to be equal to the flow leaving the node. This conservation property does not apply to the source node $(s)$ and the sink node $(t)$. Formulating a linear programming problem to maximize the overall flow departing from the source node leads us to the formulation provided below: 

\begin{array}{lll}
\text{max} & \sum_{e \in \delta^{+}(s)} f(e) - \sum_{e \in \delta^{-}(s)} f(e) & \\ \\
\text{subject to} & \sum_{e \in \delta^{-}(v)} f(e) = \sum_{e \in \delta^{+}(v)} f(e) & \forall v \in V \setminus \{s, t\}\quad \quad \text{  (flow conservation)} \\ \\
& f(e) \leq c(e) & \forall e \in E \quad \quad \text{  (capacity constraints)}\\ \\
& f(e) \geq 0 & \forall e \in E
\end{array}

For the directed graph given below, solve the max-flow problem using Pyomo


In [1]:
# import image module 
from IPython.display import Image 
  
# get the image 
Image(url="fig1.png", width=500, height=300) 

In [2]:
# Including Libraries

from pyomo.environ import *
import numpy as np
import pprint

In [3]:
# Create a Pyomo Model
model = ConcreteModel()

# Define Sets
model.v = RangeSet(0,5)

edges = [(0,1), (0,2), (1,2), (1,3), (2,1), (2,4), (3,2), (3,5), (4,3), (4,5)] # Edge tuples
model.edges = Set(initialize = edges) 

# Define parameters

cap_list = [16, 13, 10, 12, 4, 14, 9, 20, 7, 4]
capacities = {(i, j): cap_list[idx] for idx, (i, j) in enumerate(model.edges)}

model.cap = Param(model.edges, initialize = capacities)

# Define Variables
model.flow = Var(model.edges, domain= NonNegativeReals)

In [4]:
# Define objective

model.obj = Objective(expr = sum(model.flow[(i,j)] for i, j in model.edges if i ==0) -  sum(model.flow[(i,j)] for i,j in model.edges if j ==0), sense = maximize)

In [5]:
# Define constraints

# Flow conservation constraint

def flow_conservation_rule(model, v):
    if v == 0 or v ==5:
        return Constraint.Skip
    else:
        return sum(model.flow[(i,j)] for i,j in model.edges if j ==v) == sum(model.flow[(i,j)] for i,j in model.edges if i ==v)
        
model.flow_consv = Constraint(model.v, rule = flow_conservation_rule)

# Capacity constraint

def capacity_rule(model, i, j):
    return model.flow[(i,j)] <= model.cap[(i,j)]

model.capacity = Constraint(model.edges, rule = capacity_rule)

In [19]:
# Solve the model

solver = SolverFactory('gurobi')
solver.solve(model)

{'Problem': [{'Name': 'x1', 'Lower bound': 23.0, 'Upper bound': 23.0, 'Number of objectives': 1, 'Number of constraints': 14, 'Number of variables': 10, 'Number of binary variables': 0, 'Number of integer variables': 0, 'Number of continuous variables': 10, 'Number of nonzeros': 26, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Return code': '0', 'Message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Wall time': '0.0', 'Error rc': 0, 'Time': 0.3313307762145996}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [20]:
# printing the result

print("The optimal objective value is:", model.obj())

print("\nPrinting variable values:")
for i,j in model.edges:
    print("Flow through edge", (i,j), "is:" ,model.flow[(i,j)].value)

The optimal objective value is: 23.0

Printing variable values:
Flow through edge (0, 1) is: 16.0
Flow through edge (0, 2) is: 7.0
Flow through edge (1, 2) is: 4.0
Flow through edge (1, 3) is: 12.0
Flow through edge (2, 1) is: 0.0
Flow through edge (2, 4) is: 11.0
Flow through edge (3, 2) is: 0.0
Flow through edge (3, 5) is: 19.0
Flow through edge (4, 3) is: 7.0
Flow through edge (4, 5) is: 4.0
